# tf.data의 사용법
numpy package를 이용하여 간단하게 data에 해당하는 X, target에 해당하는 y를 생성하여 tf.data package의 각종 module, function을 이용한다. epoch 마다 validation data에 대해서 validation을 하는 상황을 가정


### Dataset 생성 -> Iterator 생성 -> Dataset 사용 순으로 처리함


# Template 
for문을 활용하여 model training시 data pipeline으로 아래의 function과 method를 사용하는 방법에 대한 예시
* Dataset class
 * tf.data.Dataset.from_tensor_slices로 Dataset class의 instance 새성
   * train data에 대한 Dataset class의 instance => tr_data
   * validation data에 대한 Dataset class의 instance => val_data
 * 아래와 같은 method를 활용하여 training 시 필요한 요소를 지정
   * instance의 shuffle method를 활용하면 shuffling
   * instance의 batch method를 활용하여, batch size 지정
   * for 문으로 전체 epoch를 control하므로 repeat method는 활용하지 않음
* Iterator class
 * Dataset class의 instance에서 make_initializable_iterator method로 iterator class의 instance를 생성
   * train data에 대한 iterator class의 instance, tr_iterator
   * validation data에 대한 iterator class의 instance, val_iterator
   * 주의사항으로 make_initializable_iterator method로 iterator class의 instance를 생성하는 경우, random_sees를 고정하지 않는다. 
     * random_seed를 고정하는 경우, 서로 다른 epoch의 step 별 mini-batch의 구성이 완전히 같아진다.
   * Anonymous iterator를 tf.data.Iterator.from_string_handle로 생성한다. 
     * string_handle argument에 tf.placeholder를 사용한다.
       * tr_iterator를 사용할지 val_iterator를 활용할지 조절
 

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# 전체 데이터의 개수가 12개인 임의의 데이터셋 생성
# c_는 각각의 1 D-array의 column을 하나씩 빼서 [ ]에 담음  
X = np.c_[np.arange(12), np.arange(12)]
y = np.arange(12)

print(X, X.shape, y.shape)

[[ 0  0]
 [ 1  1]
 [ 2  2]
 [ 3  3]
 [ 4  4]
 [ 5  5]
 [ 6  6]
 [ 7  7]
 [ 8  8]
 [ 9  9]
 [10 10]
 [11 11]] (12, 2) (12,)


In [3]:
# 위의 데이터를 train, validation set으로 split
X_tr=X[:8]
y_tr=y[:8]

X_val = X[8:]
y_val = y[8:]

print(X_tr.shape, y_tr.shape)
print(X_val.shape, y_val.shape)


(8, 2) (8,)
(4, 2) (4,)


In [4]:
n_epoch = 3
batch_size = 2

# 전체 step 수 : training set / batch_size
total_steps = int(X_tr.shape[0]/ batch_size)
print('epoch : {}, batch_size : {}, total_steps : {}'.format(n_epoch, batch_size, total_steps ))

epoch : 3, batch_size : 2, total_steps : 4


In [5]:
tr_data = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
# from_tensor_slices
# dataset instance를 만들고 
tr_data = tr_data.shuffle(buffer_size = 30)
tr_data = tr_data.batch(batch_size = batch_size)
# 마찬가지로 from_tensor_slices를 통해서 validation data의
# instance를 만들고 그 인스턴스를 batch 설정
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_data = val_data.batch(batch_size = batch_size)

print(tr_data)
print(val_data)

<BatchDataset shapes: ((?, 2), (?,)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((?, 2), (?,)), types: (tf.int32, tf.int32)>


# 이 방법은 Placeholder로 Dataset 객체를 만드는 경우이다.
make_initializable_iterator() 만들어야 함 feed_dict를 선택적으로 하므르써 train data, val data 나눌 수 있음 

In [6]:
#placeholder 를 사용하여 Dataset을 생성한 경우

tr_iterator = tr_data.make_initializable_iterator()
val_iterator = val_data.make_initializable_iterator()

#####
handle = tf.placeholder(dtype=tf.string)
#####

In [7]:
iterator = tf.data.Iterator.from_string_handle( string_handle = handle,
             output_shapes = tr_iterator.output_shapes,
             output_types = tr_iterator.output_types)
    
X_mb, y_mb = iterator.get_next() # iteration 등록

# 출력으로 batch_size = 2 로서 2개씩 나오는 것을 확인할 수 있다.
train_set에 대한 iteration, validation set에 대한 iteration

* 자주 사용하는 형태
```
while True:
    try: 
        께속 다음값으로 넘어갈 수 있을 때
    except: 
        끝에 도달하면
```


In [8]:

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)

                                # 이렇게 training 데이터를 뽑는 case, validation 데이터를 뽑는 case로 나눈다.
tr_handle, val_handle = sess.run([tr_iterator.string_handle(), 
                                  val_iterator.string_handle()])


# 2 개의 batch_size만큼 뽑겠다.

for epoch in range(n_epoch):
    
    print('epoch : {} training start'.format(epoch+1))
    sess.run(tr_iterator.initializer)
    n_tr_step = 0
    
    while True:
        try:
            n_tr_step+=1
            X_tmp, y_tmp = sess.run([X_mb, y_mb], feed_dict = {handle: tr_handle})
            print('step : {}'.format(n_tr_step))
            #print(X_tmp, y_tmp)
            for f, l in zip(X_tmp, y_tmp):
                print("feature: {} label {}".format(f,l))
        
        except:
            print('epoch : {} training finished'.format(epoch+1))
            break
        
    print('at epoch L {}. validation start'.format(epoch + 1))
    sess.run(val_iterator.initializer)
        
    n_val_step = 0
    while True:
        try:
            n_val_step +=1
            X_tmp, y_tmp = sess.run([X_mb, y_mb], feed_dict = {handle: val_handle})
                
            print('step : {}'.format(n_val_step))
            print(X_tmp, y_tmp)
        except:
            print('validation finished')
            break
    
    print("\n\n")

epoch : 1 training start
step : 1
feature: [6 6] label 6
feature: [5 5] label 5
step : 2
feature: [7 7] label 7
feature: [1 1] label 1
step : 3
feature: [3 3] label 3
feature: [2 2] label 2
step : 4
feature: [0 0] label 0
feature: [4 4] label 4
epoch : 1 training finished
at epoch L 1. validation start
step : 1
[[8 8]
 [9 9]] [8 9]
step : 2
[[10 10]
 [11 11]] [10 11]
validation finished



epoch : 2 training start
step : 1
feature: [1 1] label 1
feature: [5 5] label 5
step : 2
feature: [2 2] label 2
feature: [6 6] label 6
step : 3
feature: [0 0] label 0
feature: [4 4] label 4
step : 4
feature: [3 3] label 3
feature: [7 7] label 7
epoch : 2 training finished
at epoch L 2. validation start
step : 1
[[8 8]
 [9 9]] [8 9]
step : 2
[[10 10]
 [11 11]] [10 11]
validation finished



epoch : 3 training start
step : 1
feature: [1 1] label 1
feature: [4 4] label 4
step : 2
feature: [7 7] label 7
feature: [2 2] label 2
step : 3
feature: [6 6] label 6
feature: [3 3] label 3
step : 4
feature: [5 5] 

# Input pipeline

## 1. tf.data.Dataset 이라는 코드를 정의<br>
* 메모리 안의 있는 몇몇 tensor들로부터 Dataset을 만들기위해 다음과 같은 method 사용
  * tf.data.TextLineDataset(filenames)
  * tf.data.FixedLengthRecordDataset(filenames)
  * tf.data.TFRecordDataset(filenames)

## 2. Transformation 
* Dataset.map()
* Dataset.batch() : batch 사이즈를 정하는 것
* Dataset.shuffle() : 섞는 것

### 3. Iterator
* Iterator.initializer: iterator의 상태를 intialize한다.
* iterator.get_next() : 다음 iterator를 얻어오는 것


 # ● placeholder를 쓰지 않고 Dataset 객체 할당하는 경우
 make_one_shot_iterator를 사용하면 된다.

In [9]:
# 일반적인 방법
# 요것은 generator이다. 
# 리스트를 하나씩 yield 한다! 
def generator():
    for i in range(10):
        yield i

        # yield를 한 generator가 생성한 값이 텐서플로우에 생성
dataset = tf.data.Dataset.from_generator(generator, tf.float32)\
                         .make_one_shot_iterator()\
                         .get_next()
        # 완전 iterator 개념! 

with tf.Session() as sess:
    for _ in range(10): # 횟수가 넘어가면 에러가 뜬다.
        _data = sess.run(dataset)
        print(_data)

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0


In [10]:
# 우리는 데이터셋을 만들 때 라벨과 feature를 만드는데 
def generator():
    for i, j in zip(range(10, 20), range(10)):
        yield (i, j)
        
# yield를 한 generator가 생성한 값이 텐서플로우에 생성
# 두개를 yield 할 때는 ( , ) 튜플로 넣어주자.
# 그 타입으로 받아 와라라는 말
dataset = tf.data.Dataset.from_generator(generator, (tf.float32,tf.float32))\
                         .make_one_shot_iterator()\
                         .get_next()
        # 완전 iterator 개념! 

with tf.Session() as sess:
    for _ in range(10): # 횟수가 넘어가면 에러가 뜬다.
        _label, _feat = sess.run(dataset)
        print(_label, _feat)

10.0 0.0
11.0 1.0
12.0 2.0
13.0 3.0
14.0 4.0
15.0 5.0
16.0 6.0
17.0 7.0
18.0 8.0
19.0 9.0


# 데이터가 매우 많을 때 1000개?
그것에 대한 미니배치를 쉽게 할 수 있다.

.batch(batch_size) 함수를 이용하여 지정 <br>
.shuffle로 데이터를 섞을 수 있다. 인자는 셔플 사이즈이다.
(셔플하려면 일정량의 통을 가지고 있어야 하니까 통 사이즈)
얼마만큼 큐에다 넣고 섞을 지 지정한 것 


In [11]:
# 우리는 데이터셋을 만들 때 라벨과 feature를 만드는데 

def generator():
    for i, j in zip(range(10, 1100), range(1000)):
        yield (i, j)
        
# yield를 한 generator가 생성한 값이 텐서플로우에 생성
# 두개를 yield 할 때는 ( , ) 튜플로 넣어주자.
# 그 타입으로 받아 와라라는 말
dataset = tf.data.Dataset.from_generator(generator, (tf.float32,tf.float32))\
                         .batch(20)\
                         .shuffle(7777)\
                         .make_one_shot_iterator()\
                         .get_next()\
        # 완전 iterator 개념! 

with tf.Session() as sess:
    for _ in range(3): # 횟수가 넘어가면 에러가 뜬다.
        # 20개씩 세트로 리턴된다.
        _label, _feat = sess.run(dataset)
        print(_label, _feat)

[830. 831. 832. 833. 834. 835. 836. 837. 838. 839. 840. 841. 842. 843.
 844. 845. 846. 847. 848. 849.] [820. 821. 822. 823. 824. 825. 826. 827. 828. 829. 830. 831. 832. 833.
 834. 835. 836. 837. 838. 839.]
[850. 851. 852. 853. 854. 855. 856. 857. 858. 859. 860. 861. 862. 863.
 864. 865. 866. 867. 868. 869.] [840. 841. 842. 843. 844. 845. 846. 847. 848. 849. 850. 851. 852. 853.
 854. 855. 856. 857. 858. 859.]
[770. 771. 772. 773. 774. 775. 776. 777. 778. 779. 780. 781. 782. 783.
 784. 785. 786. 787. 788. 789.] [760. 761. 762. 763. 764. 765. 766. 767. 768. 769. 770. 771. 772. 773.
 774. 775. 776. 777. 778. 779.]


# 위의 결과는 배치 내부끼리는 안 섞이고 다른 배치들과는 섞이는 것을 볼 수 있다.

제너레이터 하나 정의하고 쓰는 것이 굉장히 편하다. 이미지 같은 것을 generator로 yield해서 전달하는 것을 구현할 수도 있다. 

하지만 이렇게 직접 정의하는 generator 가지고는 병목이 일어날 수 있다. API를 제공하더라도 generator에 의존하니까!

In [14]:
# 우리는 데이터셋을 만들 때 라벨과 feature를 만드는데 

def generator():
    for i, j in zip(range(10, 1100), range(1000)):
        yield (i, j)
        
# yield를 한 generator가 생성한 값이 텐서플로우에 생성
# 두개를 yield 할 때는 ( , ) 튜플로 넣어주자.
# 그 타입으로 받아 와라라는 말

#           TextLineDataset이라는 것도 있다. 
dataset = tf.data.TextLineDataset('./up_down_dataset.csv')\
                         .make_one_shot_iterator()\
                         .get_next()\
        # 완전 iterator 개념! 


# 결과는 String 형으로 파일을 읽어올 수 있다. LinebyLine
# 얘를 가져다가 디코딩을 해줄 수가 있다. 


# csv 파일에서는 missing value, 즉 빠진 값이 존재할 수 있는데
# 그것은 멀로 채울 것이냐? 에 대한 정의 record_default 


lines = tf.decode_csv(dataset,
                      record_defaults=[[0]]*11)
feature = tf.stack(lines[1:]) # 배치의 셋팅을 안했을 때
                              # 축을 넣지 않는다.
#feature = tf.stack(lines[1:], axis=1)
label = lines[0]


with tf.Session() as sess:
    _feat,_lab =sess.run([feature, label])
    print(_feat, _lab )
#     _feat, _lab = sess.run([feature, label])
#     print(_feat, _lab)

[0 1 2 3 4 5 6 7 8 9] 1


# 하나짜리는 별 신경 안써도 되지만 배치로 합칠 때는 axis를 고려해야 한다.
numpy에는 vstack이 있는데 tf에는 axis 잘따져야함

In [ ]:
dataset = tf.data.TextLineDataset('./up_down_dataset.csv')\
                         .batch(20)\
                         .make_one_shot_iterator()\
                         .get_next()\

lines = tf.decode_csv(dataset,
                      record_defaults=[[0]]*11)
feature = tf.stack(lines[1:], axis=1)
label = lines[0]

with tf.Session() as sess:
    _feat,_lab = sess.run([feature, label])
    for f, l in zip(_feat, _lab):
        print(l,f) # 1:1 매칭 시켜준다. 

In [ ]:
dataset = tf.data.TextLineDataset('./up_down_dataset.csv')\
                         .batch(20)\
                         .make_one_shot_iterator()\
                         .get_next()\

lines = tf.decode_csv(dataset,
                      record_defaults=[[0]]*11) # [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
# lines는 아래와 같은 배열이 들어있다.
'''
 그래서 세로로 배치 사이즈 만큼 원소가 잡힌다. 20개

 lines[0] = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

[array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]), #원소 20개
 array([0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9]),
 array([1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8]),
 array([2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7]),
 array([3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6]),
 array([4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5]),
 array([5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4, 5, 4]),
 array([6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3]),
 array([7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2]),
 array([8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1, 8, 1]),
 array([9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0])]
 
 '''

feature = tf.stack(lines[1:], axis=1)
label = lines[0]

# feature가 우리가 예상한 batch size의 출력으로 나온다.
with tf.Session() as sess:
    for i in range(5):
        _feat,_lab = sess.run([feature, label])
        print(_feat.shape, _lab.shape)

In [16]:
import tensorflow as tf
import numpy as np

x = np.random.sample((10, 2))
dataset = tf.data.Dataset.from_tensor_slices(x)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_element))
        except tf.errors.OutOfRangeError:
            break

[0.48818791 0.65980126]
[0.34238975 0.98514686]
[0.7840512  0.59899738]
[0.0241973  0.55126401]
[0.0772997  0.95676098]
[0.19067756 0.71681624]
[0.021248   0.04618403]
[0.1549198  0.17103496]
[0.67748982 0.47563405]
[0.32795494 0.17532543]
